# 🦌 Node Menagerie ⚡

Demonstrating the variety of Node Shapes

In [2]:
import json
import pathlib
from dataclasses import asdict

import importnb
import ipywidgets as W
import ipywidgets
import networkx as nx
import traitlets
from IPython.display import JSON, SVG

import ipyelk
import ipyelk.contrib.shapes.connectors as conn
import ipyelk.diagram.elk_export
import ipyelk.nx
import ipyelk.tools
from ipyelk import Elk
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.defs import (
    Circle,
    ConnectorDef,
    Def,
    Ellipse,
    Path,
    Point,
    RawSVG,
    Rect,
)
from ipyelk.diagram.elk_model import ElkLabel, ElkNode, ElkPort


# I want to be able to:

```Python
class Ellipse(Symbol):
    pass

class Diamond(Symbol):
    pass

class Rect(Symbol): # default
    pass

class Compartment(Symbol):
    pass




```

In [3]:
from ipyelk.diagram.layout_options.layout import ELKRectanglePacking
l = opt.LayoutAlgorithm(value=ELKRectanglePacking.identifier)
l

LayoutAlgorithm(children=(Dropdown(description='Layout Algorithm', index=4, options=(('Draw2D Layout', 'org.ec…

In [20]:
g = nx.Graph()
tree = nx.DiGraph()

record_layout = opt.OptionsWidget(
    options=[
        opt.LayoutAlgorithm(value=ELKRectanglePacking.identifier),
        opt.HierarchyHandling(),
        opt.Padding(left=0, right=0, bottom=0, top=0),
        opt.NodeSpacing(spacing=0),
        opt.EdgeNodeSpacing(spacing=0),
        opt.AspectRatio(),
        opt.ExpandNodes(activate=True),
    ]
).value

compartment_opts = opt.OptionsWidget(
    options=[
        opt.NodeLabelPlacement(),
        opt.NodeSizeConstraints()
    ]
).value
g.add_node(
    'record',
#     properties={
#         "type": "node:compartment",
#     },
    layoutOptions=record_layout
)

compartments = ["a", "b"]
for i, c in enumerate(compartments):
    g.add_node(
        c,
#         properties={
#         "type": "node:compartment",
#         },
        layoutOptions=compartment_opts
    )
    
    tree.add_edge('record', c)
    
#     if i == 0:
#         tree.add_edge('record', compartments[i])
#     if i >= 1:
#         g.add_edge(
#             compartments[i-1], compartments[i],
#             properties={
#                 "cssClasses": "hidden",
#             },
#         )
#         tree.add_edge(
#             compartments[i-1], compartments[i],
#             properties={
#                 "cssClasses": "hidden",
#             },
#         )
        
#         TODO need to add artificial edges between compartments?

# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        layouts={
            ipyelk.diagram.elk_model.ElkRoot:{
                "parents":opt.OptionsWidget(

                    options=[
                        opt.LayoutAlgorithm(),
                        opt.HierarchyHandling(),
#                         opt.Direction(value="UP"),
#                         opt.NodeLabelPlacement(),
#                         opt.ConsiderModelOrder(value="NODES_AND_EDGES"),
                    ]
                ).value,
            }
        }, 
        source=(g, tree)
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
#             "display": "none",
        }
    }
)
app

Elk(children=[HTML(value='<style>.styled-widget-140334144061200 .hidden{}</style>', layout=Layout(display='Non…

In [21]:
app.transformer.value

{'id': 'root',
 'children': [{'id': 'record',
   'children': [{'id': 'a',
     'children': [],
     'labels': [{'id': 'a_label_0_a',
       'text': 'a',
       'height': 13.015625,
       'width': 9.0}],
     'layoutOptions': {'org.eclipse.elk.nodeLabels.placement': 'H_LEFT V_TOP INSIDE',
      'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'}},
    {'id': 'b',
     'children': [],
     'labels': [{'id': 'b_label_0_b',
       'text': 'b',
       'height': 13.015625,
       'width': 8.0}],
     'layoutOptions': {'org.eclipse.elk.nodeLabels.placement': 'H_LEFT V_TOP INSIDE',
      'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'}}],
   'labels': [{'id': 'record_label_0_record',
     'text': 'record',
     'height': 13.015625,
     'width': 34.5625}],
   'layoutOptions': {'org.eclipse.elk.algorithm': 'org.eclipse.elk.rectpacking',
    'org.eclipse.elk.hierarchyHandling': 'INCLUDE_CHILDREN',
    'org.eclipse.elk.paddin

In [10]:
app.transformer.text_sizer = None
await app.transformer._refresh()

TypeError: get() takes 2 positional arguments but 3 were given

In [11]:
%debug

> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/transform.py(159)merge()
    157         d2 = {}
    158 
--> 159     cl1 = d1.get("cssClasses", "")
    160     cl2 = d2.get("cssClasses", "")
    161     cl = " ".join(sorted(set([*cl1.split(), *cl2.split()]))).strip()



ipdb>  d1


OptionsWidget()


ipdb>  d2


{}


ipdb>  c


In [22]:
g = nx.Graph()
tree = nx.DiGraph()

layout = opt.OptionsWidget(
    options=[
            opt.Direction(value="DOWN"),
#             opt.HierarchyHandling(),
        opt.NodeSizeConstraints(),
        opt.NodeLabelPlacement(horizontal="center"),
        opt.Padding(left=0, right=0, bottom=0, top=0),
        opt.NodeSpacing(spacing=0),
        opt.EdgeNodeSpacing(spacing=0),
        opt.ExpandNodes(activate=True),
    ]
).value
# layout = {}

g.add_node(
    'record',
#     properties={
#         "type": "node:compartment",
#     },
    layoutOptions=layout
)

compartments = ["a", "b"]
for i, c in enumerate(compartments):
    g.add_node(
        c,
#         properties={
#         "type": "node:compartment",
#         },
        layoutOptions=layout
    ),
    tree.add_edge('record', c)
    
#     if i == 0:
#         tree.add_edge('record', compartments[i])
    if i >= 1:
        g.add_edge(
            compartments[i-1], compartments[i],
            properties={
                "cssClasses": "hidden",
            },
        )
#         tree.add_edge(
#             compartments[i-1], compartments[i],
#             properties={
#                 "cssClasses": "hidden",
#             },
#         )
        
#         TODO need to add artificial edges between compartments?
opt.Direction

# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        layouts={
            ipyelk.diagram.elk_model.ElkRoot:{
                "parents":opt.OptionsWidget(

                    options=[
                        opt.LayoutAlgorithm(),
                        opt.HierarchyHandling(),
                        opt.Direction(value="UP"),
#                         opt.NodeLabelPlacement(),
#                         opt.ConsiderModelOrder(value="NODES_AND_EDGES"),
                    ]
                ).value,
            }
        }, 
        source=(g, tree)
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
#             "display": "none",
        }
    }
)
app

Elk(children=[HTML(value='<style>.styled-widget-140334145061904 .hidden{}</style>', layout=Layout(display='Non…

In [7]:
app.diagram.value

{'id': 'root',
 'children': [{'id': 'record',
   'children': [{'id': 'a',
     'children': [],
     'labels': [{'id': 'a_label_0_a',
       'text': 'a',
       'height': 13.015625,
       'width': 9.0}],
     'layoutOptions': {'org.eclipse.elk.direction': 'DOWN',
      'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE',
      'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE',
      'org.eclipse.elk.padding': '[top=0.0,bottom=0.0,left=0.0,right=0.0]',
      'org.eclipse.elk.spacing.nodeNode': '0.0',
      'org.eclipse.elk.edge.edgeNode': '0.0',
      'org.eclipse.elk.expandNodes': 'true'}},
    {'id': 'b',
     'children': [],
     'labels': [{'id': 'b_label_0_b',
       'text': 'b',
       'height': 13.015625,
       'width': 8.0}],
     'layoutOptions': {'org.eclipse.elk.direction': 'DOWN',
      'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE',
      'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V

In [26]:
g = nx.Graph()
tree = nx.DiGraph()

node_types = {
    'node:diamond': {},
    'node:round': {},
    "node:image":{
#         'use':"./static/base/images/favicon.ico"#/untitled_example.svg
#             "use": "https://d15omoko64skxi.cloudfront.net/wp-content/uploads/2020/12/Big-Wins-Feature-Image-Final-1200x686.jpg"
        'use':"./files/examples/untitled_example.svg"
    },
    'node:comment':{},
    "node:path": {
        'use': "M 0,0 L 0,100 L 20,30 Z"
    },
    "node:raw": {
#         "use": '<polygon points="50,0 100,50 50,100 0,50"></polygon>'
        "use": """<g><rect fill="none" height="24" width="24"/><path d="M17,15l1.55,1.55c-0.96,1.69-3.33,3.04-5.55,3.37V11h3V9h-3V7.82C14.16,7.4,15,6.3,15,5c0-1.65-1.35-3-3-3S9,3.35,9,5 c0,1.3,0.84,2.4,2,2.82V9H8v2h3v8.92c-2.22-0.33-4.59-1.68-5.55-3.37L7,15l-4-3v3c0,3.88,4.92,7,9,7s9-3.12,9-7v-3L17,15z M12,4 c0.55,0,1,0.45,1,1s-0.45,1-1,1s-1-0.45-1-1S11.45,4,12,4z"/></g>"""
    },
    "node:foreignobject": {
        "use": "<input/>"
    },
}
for n_type, attrs in node_types.items():
    g.add_node(
        n_type,
        width=100,
        height=100,
        properties = {
            "type":n_type,
            "shape": attrs,
        }
    )
    
    
tree.add_edge('node:round', "node:image")
g.add_edge("node:comment", "node:path")



# compartment experiments
g.add_node(
    'record',
    properties={
        "type": "node:compartment",
    },
    layoutOptions=opt.OptionsWidget(
        options=[
#             opt.Direction(value="DOWN"),
#             opt.HierarchyHandling(),
            opt.NodeSizeConstraints(),
            opt.NodeLabelPlacement(),
            opt.Padding(left=0, right=0, bottom=0, top=0),
            opt.NodeSpacing(spacing=0),
            opt.EdgeNodeSpacing(spacing=0),
        ]
    ).value
)

compartments = ["a", "b", "c", "d"]
for i, c in enumerate(compartments):
    g.add_node(
        c,
        properties={
        "type": "node:compartment",
        },
        layoutOptions=opt.OptionsWidget(
            options=[
#                 opt.Direction(value="LEFT"),
                opt.NodeSizeConstraints(),
                opt.NodeLabelPlacement(),
                opt.Padding(left=0, right=0, bottom=0, top=0),
                opt.NodeSpacing(spacing=0),
                opt.EdgeNodeSpacing(spacing=0),
            ]
        ).value
    ),
    
    if i == 0:
        tree.add_edge('record', compartments[i])
    if i >= 1:
        tree.add_edge(
            compartments[i-1], compartments[i],
            properties={
                "cssClasses": "hidden",
            },
        )
        
#         TODO need to add artificial edges between compartments?
opt.Direction

# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        layouts={
            ipyelk.diagram.elk_model.ElkRoot:{
                "parents":opt.OptionsWidget(

                    options=[
                        opt.LayoutAlgorithm(),
                        opt.HierarchyHandling(),
#                         opt.Direction(value="UP"),
                        opt.NodeLabelPlacement(),
#                         opt.ConsiderModelOrder(value="NODES_AND_EDGES"),
                    ]
                ).value,
            }
        }, 
        source=(g, tree)
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
#             "display": "none",
        }
    }
)
app



Elk(children=[HTML(value='<style>.styled-widget-140334149359312 .hidden{}</style>', layout=Layout(display='Non…

EdgeNodeSpacing(children=(FloatSlider(value=0.0, description='Edge Node Spacing'),))

In [10]:
app.refresh()

In [4]:
a = "werw"
a.title()

'Werw'

In [33]:
JSON(app.diagram.value)

<IPython.core.display.JSON object>

In [8]:
app.transformer.layouts

{ElkRootNode(): {'parents': {'org.eclipse.elk.algorithm': 'org.eclipse.elk.layered',
   'org.eclipse.elk.hierarchyHandling': 'INCLUDE_CHILDREN',
   'org.eclipse.elk.direction': 'DOWN'}}}

In [11]:
app.refresh()

Task exception was never retrieved
future: <Task finished coro=<ElkTransformer._refresh() done, defined at /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/transform.py:77> exception=AttributeError("'generator' object has no attribute 'items'")>
Traceback (most recent call last):
  File "/home/dfreeman6/Documents/ipyelk/py_src/ipyelk/transform.py", line 78, in _refresh
    root_node = await self.transform()
  File "/home/dfreeman6/Documents/ipyelk/py_src/ipyelk/nx/transformer.py", line 129, in transform
    visible_edges, hidden_edges = self.collect_edges()
  File "/home/dfreeman6/Documents/ipyelk/py_src/ipyelk/nx/transformer.py", line 503, in collect_edges
    closest_visible = map_visible(g, tree, attr)
  File "/home/dfreeman6/Documents/ipyelk/py_src/ipyelk/nx/nx.py", line 154, in map_visible
    if not is_hidden(tree, n, attr):
  File "/home/dfreeman6/Documents/ipyelk/py_src/ipyelk/nx/nx.py", line 90, in is_hidden
    for ancestor in nx.ancestors(tree, node):
  File "/home/dfreeman6/D

In [13]:
tree.nodes()

NodeView(('node:round', 'node:image', None, 'a', 'b', 'c', 'd'))